<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-Constants" data-toc-modified-id="Imports-and-Constants-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and Constants</a></span></li><li><span><a href="#Load-Saved-Data" data-toc-modified-id="Load-Saved-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Saved Data</a></span><ul class="toc-item"><li><span><a href="#Load-labels" data-toc-modified-id="Load-labels-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load labels</a></span></li><li><span><a href="#Load-loc_dict" data-toc-modified-id="Load-loc_dict-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load <code>loc_dict</code></a></span></li><li><span><a href="#country_indices-and-country_labels" data-toc-modified-id="country_indices-and-country_labels-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span><code>country_indices</code> and <code>country_labels</code></a></span></li><li><span><a href="#Get-folds" data-toc-modified-id="Get-folds-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Get folds</a></span></li></ul></li><li><span><a href="#General-Training-Code" data-toc-modified-id="General-Training-Code-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>General Training Code</a></span></li><li><span><a href="#OOC" data-toc-modified-id="OOC-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>OOC</a></span><ul class="toc-item"><li><span><a href="#Concatenated-MS,-NL-Features" data-toc-modified-id="Concatenated-MS,-NL-Features-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Concatenated MS, NL Features</a></span></li><li><span><a href="#Transfer-features" data-toc-modified-id="Transfer-features-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Transfer features</a></span></li></ul></li><li><span><a href="#OOC-keep_frac" data-toc-modified-id="OOC-keep_frac-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>OOC keep_frac</a></span><ul class="toc-item"><li><span><a href="#Concatenated-MS,-NL-Features" data-toc-modified-id="Concatenated-MS,-NL-Features-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Concatenated MS, NL Features</a></span></li></ul></li><li><span><a href="#Incountry" data-toc-modified-id="Incountry-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Incountry</a></span><ul class="toc-item"><li><span><a href="#Concatenated-MS,-NL-Features" data-toc-modified-id="Concatenated-MS,-NL-Features-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Concatenated MS, NL Features</a></span></li><li><span><a href="#Transfer-Features" data-toc-modified-id="Transfer-Features-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Transfer Features</a></span></li></ul></li></ul></div>

Prerequisites:
1. Run the first couple of sections from `models/baselines_dhs.ipynb` to create `data/dhs_image_hists.npz`

# Imports and Constants


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
from collections import defaultdict
from glob import glob
import pickle
import os
import re
import numpy as np
import sys
import pandas as pd

sys.path.append('../')
from batchers import batcher, dataset_constants
from models.linear_model import ridge_cv, train_linear_logo
from utils.general import load_npz

In [2]:
FOLDS = ['A', 'B', 'C', 'D', 'E']
SPLITS = ['train', 'val', 'test']
DATASET_NAME = '2012-16'
LOGS_ROOT_DIR = '../logs/'
COUNTRIES = dataset_constants.DHS_COUNTRIES

KEEPS = [0.05, 0.1, 0.25, 0.5]
SEEDS = [123, 456, 789]

In [4]:
MODEL_DIRS = {
    'resnet_ms_A': '2012-16A_18preact_ms_samescaled_b64_fc01_conv01_lr0001',
    'resnet_ms_B': '2012-16B_18preact_ms_samescaled_b64_fc001_conv001_lr0001',
    'resnet_ms_C': '2012-16C_18preact_ms_samescaled_b64_fc001_conv001_lr001',
    'resnet_ms_D': '2012-16D_18preact_ms_samescaled_b64_fc001_conv001_lr01',
    'resnet_ms_E': '2012-16E_18preact_ms_samescaled_b64_fc01_conv01_lr001',
    'resnet_nl_A': '2012-16A_18preact_nl_random_b64_fc1.0_conv1.0_lr0001',
    'resnet_nl_B': '2012-16B_18preact_nl_random_b64_fc1.0_conv1.0_lr0001',
    'resnet_nl_C': '2012-16C_18preact_nl_random_b64_fc1.0_conv1.0_lr0001',
    'resnet_nl_D': '2012-16D_18preact_nl_random_b64_fc1.0_conv1.0_lr01',
    'resnet_nl_E': '2012-16E_18preact_nl_random_b64_fc1.0_conv1.0_lr0001',
    'resnet_rgb_A': '2012-16A_18preact_rgb_same_b64_fc001_conv001_lr01',
    'resnet_rgb_B': '2012-16B_18preact_rgb_same_b64_fc001_conv001_lr0001',
    'resnet_rgb_C': '2012-16C_18preact_rgb_same_b64_fc001_conv001_lr0001',
    'resnet_rgb_D': '2012-16D_18preact_rgb_same_b64_fc1.0_conv1.0_lr01',
    'resnet_rgb_E': '2012-16E_18preact_rgb_same_b64_fc001_conv001_lr0001',

    'resnet_rgb_transfer': 'transfer_2012-16nl_nlcenter_18preact_rgb_b64_fc001_conv001_lr0001',
    'resnet_ms_transfer': 'transfer_2012-16nl_nlcenter_18preact_ms_b64_fc001_conv001_lr0001',

    'incountry_resnet_ms_A': 'DHSIncountry/incountryA_18preact_ms_samescaled_b64_fc01_conv01_lr001',
    'incountry_resnet_ms_B': 'DHSIncountry/incountryB_18preact_ms_samescaled_b64_fc1_conv1_lr001',
    'incountry_resnet_ms_C': 'DHSIncountry/incountryC_18preact_ms_samescaled_b64_fc1.0_conv1.0_lr0001',
    'incountry_resnet_ms_D': 'DHSIncountry/incountryD_18preact_ms_samescaled_b64_fc001_conv001_lr0001',
    'incountry_resnet_ms_E': 'DHSIncountry/incountryE_18preact_ms_samescaled_b64_fc001_conv001_lr0001',
    'incountry_resnet_nl_A': 'DHSIncountry/incountryA_18preact_nl_random_b64_fc1.0_conv1.0_lr0001',
    'incountry_resnet_nl_B': 'DHSIncountry/incountryB_18preact_nl_random_b64_fc1.0_conv1.0_lr0001',
    'incountry_resnet_nl_C': 'DHSIncountry/incountryC_18preact_nl_random_b64_fc1.0_conv1.0_lr0001',
    'incountry_resnet_nl_D': 'DHSIncountry/incountryD_18preact_nl_random_b64_fc1.0_conv1.0_lr0001',
    'incountry_resnet_nl_E': 'DHSIncountry/incountryE_18preact_nl_random_b64_fc01_conv01_lr001',
}

# add in keep_frac/seed dirs
KEEP_MODEL_DIRS = sorted(glob(os.path.join(LOGS_ROOT_DIR, '*keep*seed*')))

for model_dir in KEEP_MODEL_DIRS:
    model_dir = os.path.basename(model_dir)
    regex = r'2012-16(\w)_18preact_(\w+)_keep(.+)_seed(\w+)_b64.+'
    m = re.match(regex, model_dir)
    fold, bands_name, keep, seed = m.groups()
    model_name = 'resnet_{b}_{f}_keep{k}_seed{s}'.format(
        b=bands_name,
        f=fold,
        k=keep,
        s=seed)
    MODEL_DIRS[model_name] = model_dir

# Load Saved Data

## Load labels

In [6]:
def get_labels():

    scalar_float_keys = ['centerlat', 'centerlon', 'wealth', 'wealthpooled', 'wealthpooled5country']
    scalar_int_keys = ['year']
    str_keys = ['country', 'urbanrural']

    keys_to_features = {}
    for key in scalar_float_keys:
        keys_to_features[key] = tf.io.FixedLenFeature(shape=[], dtype=tf.float32)
    for key in scalar_int_keys:
        keys_to_features[key] = tf.io.FixedLenFeature(shape=[], dtype=tf.int64)
    for key in str_keys:
        keys_to_features[key] = tf.io.FixedLenFeature(shape=[], dtype=tf.string)

    ex = tf.parse_single_example(example_proto, features=keys_to_features)
    locs = np.array(tf.stack([ex['centerlat'], ex['centerlon']]))
    year = ex.get('year')
    country = ex.get('country')
    wealth = ex.get('wealth')
    wealthpooled = ex.get('wealthpooled')
    wealthpooled5country = ex.get('wealthpooled5country')
    if ex.get('urbanrural') == 'U':
        urban = 1
    else:
        urban = 0

    result = {'locs': loc, 'years': year, 'countries': country, 'urban': urban, 'wealth': wealth,
                  'wealthpooled': wealthpooled, 'wealthpooled5country': wealthpooled5country}
    return(result)

dataset = batcher.get_tfrecord_paths('2012-16','all')
dataset = dataset.map(self.process_tfrecords, num_parallel_calls=self.num_threads)
print(dataset)

0


AssertionError: 

In [5]:
# get year, locs and labels from data/dhs_wealth_index.csv

surveys_df = pd.read_csv('../data/dhs_wealth_index.csv', float_precision='high')
surveys_df.rename({'LATNUM': 'lat', 'LONGNUM': 'lon', 'URBAN_RURA': 'urban'}, axis='columns', inplace=True)
for col in ['lat', 'lon', 'wealthpooled', 'wealth', 'wealthpooled5country']:
    surveys_df[col] = surveys_df[col].astype(np.float32)
surveys_df['country'] = (surveys_df['country']
                         .str.lower()
                         .str.replace(' ', '_')
                         .str.replace("'", '_'))
locs = np.transpose(np.array([surveys_df['lat'], surveys_df['lon']]))
labels  = np.array(surveys_df['wealth'])
years = np.array(surveys_df['year'])

num_examples = len(labels)
assert np.all(np.asarray([len(labels), len(locs), len(years)]) == num_examples)
print(locs, labels, years)

[[-12.350257  13.534922]
 [-12.360865  13.551494]
 [-12.613421  13.413085]
 ...
 [-16.660612  29.850649]
 [-17.914251  30.956976]
 [-17.859114  31.797626]] [ 1.7134973   1.5453347   0.6317297  ... -0.75944644  1.1111948
 -0.55453396] [2011 2011 2011 ... 2015 2015 2015]


## Load `loc_dict`

`loc_dict` has the format:
```python
{
    (lat, lon): {
        'cluster': 1,
        'country': 'malawi',
        'country_year': 'malawi_2012',  # surveyID
        'households': 25,
        'urban': False,
        'wealth': -0.513607621192932,
        'wealthpooled': -0.732255101203918,
        'year': 2012
    }, ...
}
```

NOTE: `year` and `country_year` might differ in the year. `country_year` is the survey ID, which says which year the survey started. However, sometimes the DHS surveys cross the year-boundary, in which case `country_year` will remain the same but `year` will be the next year.

In [63]:
loc_dict_path = '../data/dhs_loc_dict.pkl'  #adapted to current directory! original:   '../data/dhs_loc_dict.pkl'
with open(loc_dict_path, 'rb') as f:
    loc_dict = pickle.load(f)

## `country_indices` and `country_labels`

`country_indices` is a dictionary that maps a country name to a sorted `np.array` of its indices
```python
{ 'malawi': np.array([ 8530,  8531,  8532, ..., 10484, 10485, 10486]), ... }
```

`country_labels` is a `np.array` that shows which country each example belongs to
```python
np.array([0, 0, 0, 0, ..., 22, 22, 22])
```
where countries are indexed by their position in `dataset_constants.DHS_COUNTRIES`

In [64]:
country_indices = defaultdict(list)  # country => np.array of indices
country_labels = np.zeros(num_examples, dtype=np.int32)  # np.array of country labels

for i, loc in enumerate(locs):
    country = loc_dict[tuple(loc)]['country']
    country_indices[country].append(i)

for i, country in enumerate(COUNTRIES):
    country_indices[country] = np.asarray(country_indices[country])
    indices = country_indices[country]
    country_labels[indices] = i

## Get folds

```python
*_folds = {
    'A': { # (ooc) list of country names, (incountry) np.array of indices
           'train': [...],
           'val': [...],
           'test': [...],
    }, ...
}
```

In [65]:
ooc_folds = {
    f: {split: dataset_constants.SURVEY_NAMES[f'2009-17{f}'][split] for split in SPLITS}
    for f in FOLDS
}

In [66]:
with open('../data/dhs_incountry_folds.pkl', 'rb') as f:
    incountry_folds = pickle.load(f)

incountry_group_labels = np.zeros(num_examples, dtype=np.int32)
for i, fold in enumerate(FOLDS):
    test_indices = incountry_folds[fold]['test']
    incountry_group_labels[test_indices] = i

# General Training Code

In [67]:
def get_indices_for_countries(countries):
    indices =  np.sort(np.concatenate([
        country_indices[country] for country in countries
    ]))
    return indices

In [68]:
def countries_to_nums(countries):
    '''
    Args
    - countries: list or set of str, names of countries

    Returns: nums, list of int
    '''
    nums = []
    for c in countries:
        num = COUNTRIES.index(c)
        nums.append(num)
    return nums

# OOC

In [69]:
def ridgecv_ooc_wrapper(model_name, savedir):
    features_dict = {}
    for f in FOLDS:
        model_fold_name = f'{model_name}_{f}'
        model_dir = MODEL_DIRS[model_fold_name]
        npz = load_npz(os.path.join(LOGS_ROOT_DIR, 'DHS_OOC', model_dir, 'features.npz'),
                       check={'labels': labels})
        features = npz['features']
        for country in dataset_constants.SURVEY_NAMES[f'2012_16{f}']['test']:
            features_dict[country] = features

    ridge_cv(
        features=features_dict,
        labels=labels,
        group_labels=country_labels,
        group_names=COUNTRIES,
        do_plot=True,
        savedir=savedir,
        save_weights=True,
        save_dict=dict(locs=locs))

In [71]:
model_name = 'resnet_ms'
savedir = '../logs/dhs_resnet/ms/'
ridgecv_ooc_wrapper(model_name, savedir) #tbd: cell is not capable of creatingand saving dictionarys of the models with feature.npz (above cell line 6) 

FileNotFoundError: [Errno 2] No such file or directory: './logs/DHS_OOC\\2009-17A_18preact_ms_samescaled_b64_fc01_conv01_lr0001\\features.npz'

In [ ]:
model_name = 'resnet_nl'
savedir = '../logs/dhs_resnet/nl/'
ridgecv_ooc_wrapper(model_name, savedir)

 #%% md

## Concatenated MS, NL Features

In [ ]:
def ridgecv_ooc_concat_wrapper(model_names, savedir):
    features_dict = {}
    for f in FOLDS:
        concat_features = []  # list of np.array, each shape [N, D_i]
        for model_name in model_names:
            model_dir = MODEL_DIRS[f'{model_name}_{f}']
            npz = load_npz(os.path.join(LOGS_ROOT_DIR, 'DHS_OOC', model_dir, 'features.npz'),
                           check={'labels': labels})
            concat_features.append(npz['features'])
        concat_features = np.concatenate(concat_features, axis=1) # shape [N, D_1 + ... + D_m]
        for country in dataset_constants.SURVEY_NAMES[f'2009-17{f}']['test']:
            features_dict[country] = concat_features

    ridge_cv(
        features=features_dict,
        labels=labels,
        group_labels=country_labels,
        group_names=COUNTRIES,
        do_plot=True,
        savedir=savedir,
        save_weights=True,
        save_dict=dict(locs=locs),
        verbose=True)

In [ ]:
model_names = ['resnet_ms', 'resnet_nl']
savedir = os.path.join(LOGS_ROOT_DIR, 'dhs_resnet', 'msnl_concat')
ridgecv_ooc_concat_wrapper(model_names, savedir)

## Transfer features

In [ ]:
def ridgecv_ooc_transfer_wrapper(model_name, savedir):
    model_dir = MODEL_DIRS[model_name]
    features = load_npz(os.path.join(LOGS_ROOT_DIR, model_dir, 'features.npz'),
                        check={'labels': labels})['features']
    ridge_cv(
        features=features,
        labels=labels,
        group_labels=country_labels,
        group_names=COUNTRIES,
        savedir=savedir,
        save_weights=False,
        save_dict=dict(locs=locs),
        verbose=True)

In [ ]:
model_name = 'resnet_rgb_transfer'
savedir = '../logs/dhs_resnet/rgb_transfer/'
ridgecv_ooc_transfer_wrapper(model_name, savedir)

In [ ]:
model_name = 'resnet_ms_transfer'
savedir = '../logs/dhs_resnet/ms_transfer/'
ridgecv_ooc_transfer_wrapper(model_name, savedir)

# OOC keep_frac

In [ ]:
def get_split_idxs(dataset: str):
    train_tfrecord_paths = batcher.get_tfrecord_paths(dataset, 'train')
    val_tfrecord_paths = batcher.get_tfrecord_paths(dataset, 'val')
    test_tfrecord_paths = batcher.get_tfrecord_paths(dataset, 'test')
    all_tfrecord_paths = batcher.get_tfrecord_paths(dataset, 'all')

    path_to_idx = {path: idx for idx, path in enumerate(all_tfrecord_paths)}
    train_idxs = [path_to_idx[path] for path in train_tfrecord_paths]
    val_idxs = [path_to_idx[path] for path in val_tfrecord_paths]
    test_idxs = [path_to_idx[path] for path in test_tfrecord_paths]
    return train_idxs, val_idxs, test_idxs

def get_keep_indices(keep_frac: float, seed: int, dataset: str, test_country: str):
    '''
    Args
    - keep_frac: float, fraction of non-test-country data to use for training
    - seed: int
    - dataset: str, one of the keys of dataset_constants.SIZES[dataset]
    - test_country: str
    '''
    train_idxs, val_idxs, test_idxs = get_split_idxs(dataset)
    test_country_idxs = get_indices_for_countries([test_country]).tolist()
    test_other_idxs = sorted(set(test_idxs) - set(test_country_idxs))  # sort for determinism
    
    num_train = int(dataset_constants.SIZES[dataset]['train'] * keep_frac)
    num_val = int(dataset_constants.SIZES[dataset]['val'] * keep_frac)
    num_test = int(len(test_other_idxs) * keep_frac)

    np.random.seed(seed)
    train_idxs = np.random.choice(train_idxs, size=num_train, replace=False)
    val_idxs = np.random.choice(val_idxs, size=num_val, replace=False)
    test_other_idxs = np.random.choice(test_other_idxs, size=num_test, replace=False)

    return np.sort(np.concatenate([train_idxs, val_idxs, test_other_idxs, test_country_idxs]))

In [ ]:
def run_ridgecv_keep(model_name, labels, locs, country_labels, folds, keep_frac, seed, savedir):
    '''
    For every country C (the test country):
      1. uses leave-one-country-out CV on all other countries
         to tune ridge model alpha parameter
      2. using best alpha, trains ridge model on all countries except C
      3. runs trained ridge model on C
    Saves predictions for each country on test.

    Args
    - model_name: str, format 'resnet_{bands}', e.g. 'resnet_ms'
    - labels: np.array, shape [num_examples]
    - locs: np.array, shape [num_examples, 2]
    - country_labels: np.array, shape [num_examples]
    - folds: dict, fold (str) => dict
    - keep_frac: float, fraction of non-test-country data to use for training
    - seed: int
    - savedir: str
    '''
    test_preds = np.zeros(num_examples, dtype=np.float32)
    all_countries_set = set(COUNTRIES)

    for f in FOLDS:
        print('Fold:', f)
        model_fold_name = f'{model_name}_{f}'
        model_dir = MODEL_DIRS[model_fold_name]
        model_dir = model_dir.replace('b64', f'keep{keep_frac}_seed{seed}_b64')

        npz = load_npz(
            os.path.join(LOGS_ROOT_DIR, 'DHS_OOC', model_dir, 'features.npz'),
            check=dict(labels=labels, locs=locs))
        features = npz['features']
        dataset = '2009-17' + f

        for test_country in folds[f]['test']:
            print('test country:', test_country)
            keep_subset_indices = get_keep_indices(
                keep_frac=keep_frac,
                seed=seed,
                dataset=dataset,
                test_country=test_country)

            test_country_set = {test_country}
            cv_countries_set = all_countries_set - test_country_set
            test_indices = get_indices_for_countries(test_country_set)
            test_preds[test_indices] = train_linear_logo(
                features=features[keep_subset_indices],
                labels=labels[keep_subset_indices],
                group_labels=country_labels[keep_subset_indices],
                cv_groups=countries_to_nums(cv_countries_set),
                test_groups=countries_to_nums(test_country_set),
                plot=False,
                group_names=COUNTRIES)

    # save preds on the test set
    os.makedirs(savedir, exist_ok=True)
    npz_path = os.path.join(savedir, 'test_preds_keep{k}_seed{s}.npz'.format(k=keep_frac, s=seed))
    print('Saving preds to:', savedir)
    np.savez_compressed(npz_path, test_preds=test_preds, labels=labels, locs=locs)

In [ ]:
model_name = 'resnet_ms'
savedir = os.path.join(LOGS_ROOT_DIR, 'dhs_resnet', 'ms')

for keep_frac in KEEPS:
    for seed in SEEDS:
        print('-------------------------------------------')
        print(f'---------- keep: {keep_frac:0.02g}, seed: {seed} ----------')
        print('-------------------------------------------')
        run_ridgecv_keep(
            model_name=model_name,
            labels=labels,
            locs=locs,
            country_labels=country_labels,
            folds=ooc_folds,
            keep_frac=keep_frac,
            seed=seed,
            savedir=savedir)

In [ ]:
model_name = 'resnet_nl'
savedir = os.path.join(LOGS_ROOT_DIR, 'dhs_resnet', 'nl')

for keep_frac in KEEPS:
    for seed in SEEDS:
        print('-------------------------------------------')
        print(f'---------- keep: {keep_frac:0.02g}, seed: {seed} ----------')
        print('-------------------------------------------')
        run_ridgecv_keep(
            model_name=model_name,
            labels=labels,
            locs=locs,
            country_labels=country_labels,
            folds=ooc_folds,
            keep_frac=keep_frac,
            seed=seed,
            savedir=savedir)

## Concatenated MS, NL Features

In [ ]:
def run_ridgecv_keep_concat(model_names, concat_model_name, labels, locs,
                            country_labels, folds, keep_frac, seed):
    '''
    For every country C (the test country):
      1. uses leave-one-country-out CV on all other countries
         to tune ridge model alpha parameter
      2. using best alpha, trains ridge model on all countries except C
      3. runs trained ridge model on C
    Saves predictions for each country on test.

    Args
    - model_names: list of str
    - concat_model_name: str
    - labels: np.array, shape [num_examples]
    - locs: np.array, shape [num_examples, 2]
    - country_labels: np.array, shape [num_examples]
    - folds: dict, fold (str) => dict
    - keep_frac: float, fraction of non-test-country data to use for training
    - seed: int
    '''
    test_preds = np.zeros(num_examples, dtype=np.float32)
    all_countries_set = set(COUNTRIES)

    for f in FOLDS:
        print('Fold:', f)
        concat_features = []  # list of np.array, each shape [N, D_i]
        for model_name in model_names:
            model_fold_name = f'{model_name}_{f}'
            model_dir = MODEL_DIRS[model_fold_name].replace('b64', f'keep{keep_frac}_seed{seed}_b64')
            npz = load_npz(
                os.path.join(LOGS_ROOT_DIR, 'DHS_OOC', model_dir, 'features.npz'),
                check=dict(labels=labels, locs=locs))
            concat_features.append(npz['features'])
        concat_features = np.concatenate(concat_features, axis=1) # shape [N, D_1 + ... + D_m]
        dataset = '2009-17' + f

        do_plot = True
        for test_country in folds[f]['test']:
            print('test country:', test_country)
            keep_subset_indices = get_keep_indices(
                keep_frac=keep_frac,
                seed=seed,
                dataset=dataset,
                test_country=test_country)

            test_country_set = {test_country}
            cv_countries_set = all_countries_set - test_country_set
            test_indices = get_indices_for_countries(test_country_set)
            test_preds[test_indices] = train_linear_logo(
                features=concat_features[keep_subset_indices],
                labels=labels[keep_subset_indices],
                group_labels=country_labels[keep_subset_indices],
                cv_groups=countries_to_nums(cv_countries_set),
                test_groups=countries_to_nums(test_country_set),
                plot=do_plot,
                group_names=COUNTRIES)
            do_plot = False

    # save preds on the test set
    log_dir = os.path.join(LOGS_ROOT_DIR, 'dhs_resnet', concat_model_name)
    os.makedirs(log_dir, exist_ok=True)
    npz_path = os.path.join(log_dir, 'test_preds_keep{k}_seed{s}.npz'.format(k=keep_frac, s=seed))
    np.savez_compressed(npz_path, test_preds=test_preds, locs=locs)

In [ ]:
for keep_frac in KEEPS:
    for seed in SEEDS:
        print('-------------------------------------------')
        print(f'---------- keep: {keep_frac:0.02g}, seed: {seed} ----------')
        print('-------------------------------------------')
        run_ridgecv_keep_concat(
            model_names=['resnet_ms', 'resnet_nl'],
            concat_model_name='msnl_concat',
            labels=labels,
            locs=locs,
            country_labels=country_labels,
            folds=ooc_folds,
            keep_frac=keep_frac,
            seed=seed)

# Incountry

In [ ]:
def ridgecv_incountry_wrapper(model_name, savedir):
    features_dict = {}
    for f in FOLDS:
        model_fold_name = f'{model_name}_{f}'
        model_dir = MODEL_DIRS[model_fold_name]
        npz = load_npz(os.path.join(LOGS_ROOT_DIR, model_dir, 'features.npz'),
                       check={'labels': labels})
        features_dict[f] = npz['features']

    ridge_cv(
        features=features_dict,
        labels=labels,
        group_labels=incountry_group_labels,
        group_names=FOLDS,
        savedir=savedir,
        save_weights=True,
        verbose=True)

In [ ]:
model_name = 'incountry_resnet_ms'
savedir = '../logs/dhs_resnet/incountry_ms/'
ridgecv_incountry_wrapper(model_name, savedir)

In [ ]:
model_name = 'incountry_resnet_nl'
savedir = '../logs/dhs_resnet/incountry_nl/'
ridgecv_incountry_wrapper(model_name, savedir)

## Concatenated MS, NL Features

In [ ]:
def ridgecv_incountry_concat_wrapper(model_names, savedir):
    features_dict = {}
    for i, f in enumerate(FOLDS):
        concat_features = []  # list of np.array, each shape [N, D_i]
        for model_name in model_names:
            model_dir = MODEL_DIRS[f'{model_name}_{f}']
            npz = load_npz(os.path.join(LOGS_ROOT_DIR, model_dir, 'features.npz'),
                           check={'labels': labels})
            concat_features.append(npz['features'])
        concat_features = np.concatenate(concat_features, axis=1) # shape [N, D_1 + ... + D_m]
        features_dict[f] = concat_features

    ridge_cv(
        features=features_dict,
        labels=labels,
        group_labels=incountry_group_labels,
        group_names=FOLDS,
        savedir=savedir,
        save_weights=True,
        verbose=True)

In [ ]:
model_names = ['incountry_resnet_ms', 'incountry_resnet_nl']
savedir = os.path.join(LOGS_ROOT_DIR, 'dhs_resnet', 'incountry_msnl_concat')
ridgecv_incountry_concat_wrapper(model_names, savedir)

## Transfer Features

In [ ]:
def ridgecv_incountry_transfer_wrapper(model_name, savedir):
    model_dir = MODEL_DIRS[model_name]
    features = load_npz(os.path.join(LOGS_ROOT_DIR, model_dir, 'features.npz'),
                        check={'labels': labels})['features']
    ridge_cv(
        features=features,
        labels=labels,
        folds=incountry_folds,
        savedir=savedir,
        save_weights=False)

In [ ]:
model_name = 'resnet_rgb_transfer'
savedir = '../logs/dhs_resnet/incountry_rgb_transfer/'
ridgecv_incountry_transfer_wrapper(model_name, savedir)

In [ ]:
model_name = 'resnet_ms_transfer'
savedir = '../logs/dhs_resnet/incountry_ms_transfer/'
ridgecv_incountry_transfer_wrapper(model_name, savedir)